In [3]:
import re
import pandas as pd

# Helper functions
def clean_question_text(raw_question):
    raw_question = str(raw_question)
    raw_question = raw_question.strip()
    raw_question = re.sub(r'^.*?(\d+\.)', r'\1', raw_question)
    clean_text = re.sub(r'\\[a-zA-Z]+\{.*?\}', '', raw_question)
    #clean_text = re.sub(r'\\', '', clean_text)
    clean_text = re.sub(r'(?<!\{)[}]', '', clean_text)
    clean_text = re.sub(r'\s+', ' ', clean_text).strip()
    clean_text = re.sub(r'^\d+\.\s*', '', clean_text)
    return clean_text

def clean_options(raw_options):
    cleaned_options = []
    raw_options = str(raw_options)
    options_list = raw_options.split('% Option')
    for option in options_list:
        match = re.search(r'\((.*?)\)\s*\\\(.*?\\\)', option)
        if match:
            original_label = match.group(1)
            latex_expr = re.search(r'\\\(.*?\\\)', option).group(0)
            #latex_expr_cleaned = latex_expr.replace('\\', '').replace('\\)', '').strip()
            cleaned_option = f'{latex_expr}'
            cleaned_options.append(cleaned_option)
    return cleaned_options

def create_option_columns_with_labels(cleaned_options):
    # Create a dictionary to hold the columns dynamically
    options_dict = {}

    for i, option in enumerate(cleaned_options, start=1):
        # Assign Option 1, Option 2, etc., based on position
        column_name = f"Option_{i}_en"  # Dynamic column naming
        options_dict[column_name] = option

    # Convert the dictionary into a DataFrame with one row
    options_df = pd.DataFrame([options_dict])
    return options_df


def clean_correct_answer(raw_answer):
    raw_answer = raw_answer.strip()
    raw_answer = str(raw_answer)
    match = re.search(r'\\textbf{Correct Answer:} (.*?\\\(.*?\\\))', raw_answer)
    if match:
        answer_text = match.group(1)
        option_match = re.search(r'\((.*?)\)', answer_text)
        latex_expr_match = re.search(r'\\\(.*?\\\)', answer_text)
        if option_match and latex_expr_match:
            option = option_match.group(0)
            latex_expr = latex_expr_match.group(0)
            #latex_expr_cleaned = latex_expr.replace('\\(', '').replace('\\)', '').replace("\\", "")
            final_answer = f'{option} {latex_expr}'
            return final_answer.strip()
    return None

def clean_solution(raw_solution):
    raw_solution = str(raw_solution)
    raw_solution = raw_solution.strip()
    cleaned_solution = re.sub(r'.*?% Solution\s*\\noindent\s*\\textbf{Solution:}', '', raw_solution)
    cleaned_solution = re.sub(r'\n', '', cleaned_solution)
    cleaned_solution = re.sub(r'\s+', ' ', cleaned_solution).strip()
    return cleaned_solution

def clean_quicktip(raw_quicktip):
    raw_quicktip = str(raw_quicktip)
    cleaned_quicktip = re.sub(r'^% Quick Tip\s*', '', raw_quicktip)
    cleaned_quicktip = re.sub(r'.*?{quicktipbox}\s*', '', cleaned_quicktip)
    cleaned_quicktip = re.sub(r'\\end{quicktipbox}.*', '', cleaned_quicktip)
    #tip = re.sub(r'\\', '', cleaned_quicktip)
    cleaned_quicktip = re.sub(r'\s+', ' ', cleaned_quicktip).strip()
    return cleaned_quicktip

In [4]:
# Main function
def extract_latex_to_dataframe(latex_code):
    # Define regex patterns
    question_text_pattern = r'% Question text(.*?)% Option'
    option_pattern = r'(% Option.*?)(?=% Correct Answer)'
    correct_answer_pattern = r'% Correct Answer(.*?)% Solution'
    solution_pattern = r'(% Solution.*?)(?=% Quick Tip)'
    quick_tip_pattern = r'(% Quick Tip.*?)(?=%|$)'

    # Extract sections using regex
    question_texts = re.findall(question_text_pattern, latex_code, re.DOTALL)
    options = re.findall(option_pattern, latex_code, re.DOTALL)
    correct_answers = re.findall(correct_answer_pattern, latex_code, re.DOTALL)
    solutions = re.findall(solution_pattern, latex_code, re.DOTALL)
    quick_tips = re.findall(quick_tip_pattern, latex_code, re.DOTALL)

    # Create a DataFrame
    data = []

    for i, raw_question in enumerate(question_texts):
        question_text = clean_question_text(raw_question)

        # Process options
        raw_options = options[i] if i < len(options) else None
        cleaned_options = clean_options(raw_options) if raw_options else []
        options_df = create_option_columns_with_labels(cleaned_options)

        # Process correct answer
        raw_correct_answer = correct_answers[i] if i < len(correct_answers) else None
        correct_answer = clean_correct_answer(raw_correct_answer) if raw_correct_answer else None

        # Process solution
        raw_solution = solutions[i] if i < len(solutions) else None
        solution = clean_solution(raw_solution) if raw_solution else None

        # Process quick tip
        raw_quick_tip = quick_tips[i] if i < len(quick_tips) else None
        quick_tip = clean_quicktip(raw_quick_tip) if raw_quick_tip else None

        # Combine all into a single row
        '''
        row = {
            'S.no': i + 1,
            'Question text': question_text,
            'Correct Answer': correct_answer,
            'Solution': solution,
            'Quick Tip': quick_tip,
        }

        # Add options to the row
        for col in options_df.columns:
            row[col] = options_df[col].iloc[0]'''
        row = {
            'S.no': i + 1,
            'Question text': question_text,
        }

        # Add options to the row
        for col in options_df.columns:
            row[col] = options_df[col].iloc[0]

        # Add the remaining columns in the desired order
        row['Correct Answer'] = correct_answer
        row['Solution'] = solution
        row['Quick Tip'] = quick_tip
        data.append(row)

    # Create a DataFrame from the data
    df = pd.DataFrame(data)
    return df

In [49]:
latex_code = r'''



% Question text
\noindent \textbf{1. If gcd(m, n) = 1 and $1^2-2^2+3^2-4^2+\cdots+(2021)^2-(2022)^2+(2023)^2 = 1012 m^2n$, then $m^2-n^2$ is equal to:} \\
\textbf{Options:} \\
% Option 
(1) \(180\) \\
% Option 
(2) \(220\) \\
% Option 
(3) \(200\) \\
% Option 
(4) \(240\) \\
% Correct Answer
\textbf{Correct Answer:} (4) \(240\) \\
% Solution
\noindent \textbf{Solution:} \\
\textbf{Step 1: Simplify the series.} \\
Using the properties of sum of squares and arithmetic sequences:
\begin{align*}
    S & = 1^2 - 2^2 + 3^2 - 4^2 + \cdots + 2021^2 - 2022^2 + 2023^2 \\
      & = (1^2 + 3^2 + 5^2 + \cdots + 2023^2) - (2^2 + 4^2 + 6^2 + \cdots + 2022^2).
\end{align*}
The sum of squares of odd and even numbers up to 2023 and 2022, respectively, can be evaluated separately.

\textbf{Step 2: Use the formula for the sum of squares.} \\
\begin{align*}
    \text{Sum of odd squares} & = \left(\frac{n(2n+1)(2n-1)}{3}\right)_{n=1012}, \\
    \text{Sum of even squares} & = \left(\frac{n(n+1)(2n+1)}{3}\right)_{n=1011}.
\end{align*}

\textbf{Step 3: Calculate the final expression.} \\
After calculating the above series, equating them to   $1012 m^2n$ gives:
\[
(2023)(2023 - 1011) = (1012) mn \implies (2023)(1012) = (1012) mn \implies mn = 2023.
\]
Assuming $m$ and $n$ are coprime integers, we can further solve:
\[
m^2 - n^2 = (m+n)(m-n) = (17+7)(17-7) = 24 \cdot 10 = 240.
\]

\textbf{Final Answer:} 240.
% Quick Tip
\begin{quicktipbox}
Using properties of sequences and algebraic manipulation significantly simplifies complex series calculations.
\end{quicktipbox}

\vspace{0.5cm}
\hrule
\vspace{0.5cm}

% Question text
\noindent \textbf{2. The area bounded by the curves $y = |x - 1| + |x - 2|$ and $y = 3$ is equal to:} \\
\textbf{Options:} \\
% Option 
(1) \(5\) \\
% Option 
(2) \(4 \)\\
% Option 
(3) \(6\) \\
% Option 
(4) \(3 \)\\
% Correct Answer
\textbf{Correct Answer:} (2) \(4 \) \\
% Solution
\noindent \textbf{Solution:} \\

\textbf{Step 1: Understand the geometric interpretation.} \\
The equation $y = |x-1| + |x-2|$ forms a V-shaped curve with a vertex at $x = 1.5$. It intersects $y = 3$ at points where:
\[
|x-1| + |x-2| = 3.
\]
Solving this, we find intersections at $x = 1$ and $x = 2$.

\textbf{Step 2: Calculate the area.} \\
The area under $y=3$ from $x=1$ to $x=2$ above the V-shaped curve is a rectangle minus the area under the curve from $x=1$ to $x=2$. 
\[
A = \text{Area under } y = 3 - \text{Area under } y = |x-1| + |x-2|.
\]
Calculating these gives:
\[
A = \frac{1}{2}[1+3] [2]
\]
\[A= \frac{8}{2}\]
\[A = 4\]

The actual computation will depend on the correct interpretation of the curves which might be misstated here without visual confirmation.

\textbf{Final Answer:} 4 (as per calculated area and assuming symmetry).
% Quick Tip
\begin{quicktipbox}
For areas between curves, integrate the difference between the upper and lower function over the intersected range.
\end{quicktipbox}

\vspace{0.5cm}
\hrule
\vspace{0.5cm}

% Question text
\noindent \textbf{3. For the system of equations $x + y + z = 6$, $x + 2y + \alpha z = 10$, $x + 3y + 5z = \beta$, which one of the following is NOT true:} \\
\textbf{Options:} \\
% Option 
(1) \(System has a unique solution for $\alpha$ = $3$, $\beta \neq 14$. \)\\
% Option 
(2) \(System has a unique solution for $\alpha = -3$, $\beta = 14$. \)\\
% Option 
(3) \(System has no solution for $\alpha = 3$, $\beta = 24$.\) \\
% Option 
(4) \(System has infinitely many solutions for $\alpha = 3$, $\beta = 14$.\) \\
% Correct Answer
\textbf{Correct Answer:} (1) \(System has a unique solution for $\alpha$ = $3$, $\beta \neq 14$. \) \\
% Solution
\noindent \textbf{Solution:} \\

\textbf{Step 1: Calculate the determinant $\Delta$.} \\
\[
\Delta = \begin{vmatrix}
1 & 1 & 1 \\
1 & 2 & \alpha \\
1 & 3 & 5
\end{vmatrix}
= (10 - 3\alpha) - (5 - \alpha) + (3 - 2) = 6 - 2\alpha.
\]
\textbf{Step 2: Solve for $\Delta_x$, $\Delta_y$, and $\Delta_z$.} \\
Using Cramer's rule, substitute into the determinant to find $\Delta_x$, $\Delta_y$, and $\Delta_z$. For a unique solution, $\Delta \neq 0$. For no solution or infinitely many solutions, $\Delta = 0$ but $\Delta_x$, $\Delta_y$, or $\Delta_z$ must not all be zero.

\[
\Delta x &= 
\begin{vmatrix}
6 & 1 & 1 \\
10 & 2 & a \\
\beta & 3 & 5
\end{vmatrix} \\
&= 6(10-3a)-(50-a13)+(30-2\beta) \\
&= 40-18\alpha+\alpha\beta-2\beta \\

\Delta y &= 
\begin{vmatrix}
1 & 6 & 1 \\
1 & 10 & a \\
1 & \beta & 5
\end{vmatrix} \\
&= (50-\alpha\beta)-6(5-\alpha)+(\beta-10) \\
&= 10+6a+\beta-a\beta \\

\Delta z &= 
\begin{vmatrix}
1 & 1 & 6 \\
1 & 2 & 10 \\
1 & 3 & \beta
\end{vmatrix} \\
&= (2\beta-30)-(\beta-10)+6(1) \\
&= \beta-14
\]\\\\

\textbf{Step 3: Evaluate the conditions.} \\
Solving the determinants shows that when $\alpha = 3$, $\Delta = 0$, indicating potentially infinitely many or no solutions. Solving further confirms whether $\beta$ affects these outcomes.

\textbf{Final Answer:} The statement "System has a unique solution for $\alpha = 3$, $\beta \neq 14$" is NOT true.
% Quick Tip
\begin{quicktipbox}
Cramer's rule is pivotal for determining the nature of solutions in a system of linear equations based on the determinant of the coefficient matrix.
\end{quicktipbox}
\vspace{0.5cm}
\hline
\vspace{0.5cm}




% Question text
\noindent \textbf{4. Among the statements:}
\begin{itemize}
    \item (S1): $(p \Rightarrow q) \vee ((\sim p) \wedge q)$ is a tautology
    \item (S2): $(q \Rightarrow p) \Rightarrow ((\sim p) \wedge q)$ is a contradiction
\end{itemize}
\textbf{Options:}
\begin{enumerate}
% Option 
\(Only (S2) is True\)
% Option 
\(Only (S1) is True\)
% Option 
\(Neither (S1) and (S2) is True\)
% Option 
\(Both (S1) and (S2) are True\)
\end{enumerate}

% Correct Answer
\textbf{Correct Answer:} (3) \(Neither (S1) and (S2) is True\)\\
% Solution
\noindent \textbf{Solution:} \\

\textbf{Step 1: Analyze S1.} \\
Construct the truth table for $(p \Rightarrow q) \vee ((\sim p) \wedge q)$:

\[
\begin{array}{|c|c|c|c|c|c|}
\hline
P & Q & \sim P & (\sim P) \wedge Q & P \Rightarrow Q & (P \Rightarrow Q) \vee ((\sim P) \wedge Q) \\
\hline
T & T & F & F & T & T \\
T & F & F & F & F & F \\
F & T & T & T & T & T \\
F & F & T & F & T & T \\
\hline
\end{array}
\]

\textbf{Conclusion for S1:} $(p \Rightarrow q) \vee ((\sim p) \wedge q)$ is not true for all cases; hence it is \textbf{not a tautology}.

\textbf{Step 2: Analyze S2.} \\
Construct the truth table for $(q \Rightarrow p) \Rightarrow ((\sim p) \wedge q)$:

\[
\begin{array}{|c|c|c|c|c|c|}
\hline
P & Q & q \Rightarrow p & \sim P & (\sim P) \wedge Q & (q \Rightarrow p) \Rightarrow ((\sim P) \wedge Q) \\
\hline
T & T & T & F & F & F \\
T & F & T & F & F & F \\
F & T & F & T & T & T \\
F & F & F & T & F & T \\
\hline
\end{array}
\]

\textbf{Conclusion for S2:} $(q \Rightarrow p) \Rightarrow ((\sim p) \wedge q)$ is not false for all cases; hence it is \textbf{not a contradiction}.

\textbf{Final Answer:} Neither (S1) nor (S2) is true. (Option 3)
% Quick Tip
\begin{quicktipbox}
Truth tables are essential for analyzing logical statements. A tautology is true in all cases, while a contradiction is false in all cases.
\end{quicktipbox}

\vspace{0.5cm}
\hrule
\vspace{0.5cm}
% Question text
\noindent \textbf{5. Evaluate the limit} \\
\[lim_{n->\infty} \{(2^{\frac{1}{2}}-2^{\frac{1}{3}})(2^{\frac{1}{2}}-2^{\frac{1}{5}})...(2^{\frac{1}{2}}-2^{\frac{1}{2n+1}})\]

\textbf{Options:} \\
% Option 
(1) \( \frac{1}{\sqrt{2}} \) \\
% Option 
(2) \( \sqrt{2} \) \\
% Option 
(3) \(1\) \\
% Option 
(4) \(0 \)\\
% Correct Answer
\textbf{Correct Answer:} (4) \(0 \) \\
% Solution
\noindent \textbf{Solution:} \\

\textbf{Step 1: Analyze the behavior of each term in the product.} \\
The general term in the product is \( \frac{1}{2^2 - 2^3} \), and as \( n \) approaches infinity, the terms become increasingly small.

\textbf{Step 2: Apply the Sandwich Theorem.} \\
The terms are bounded between 0 and 1, and as \( n \to \infty \), the product approaches 0.

\[

P = lim_{n->\infty} (2^{\frac{1}{2}}-2^{\frac{1}{3}})(2^{\frac{1}{2}}-2^{\frac{1}{5}})... (2^{\frac{1}{2}}-2^{\frac{1}{2n+1}})\]

\[Let  
2^{\frac{1}{2}}-2^{\frac{1}{3}}  --> Smallest\]
\[2^{\frac{1}{2}}-2^{\frac{1}{2n+1}} --> Largest\]

\[Sandwich th.
(2^{\frac{1}{2}}-2^{\frac{1}{3}})^n <= P <= (2^{\frac{1}{2}}-2^{\frac{1}{2n+1}})^n \]


\[lim_{n->\infty} (2^{\frac{1}{2}}-2^{\frac{1}{3}})^n = 0\]

\[lim_{n->\infty} (2^{\frac{1}{2}}-2^{\frac{1}{2n+1}})^n = 0\]

 P = 0



\]

\textbf{Final Answer:} The limit of the entire product is \( 0 \).
% Quick Tip
\begin{tcolorbox}[colback=white, colframe=black, colbacktitle=white, coltitle=black,
title=QuickTip, width=\textwidth, halign=left]
The Sandwich Theorem is helpful when determining limits of products or functions that approach zero.
\end{tcolorbox}

\vspace{0.5cm}
\hline
\vspace{0.5cm}

% Question text
\noindent \textbf{6. Let P be a square matrix such that \( P^2 = I - P \). For \( \alpha, \beta, \gamma, \delta \in \mathbb{N} \), if}

\[
P^\alpha + P^\beta = \gamma I - 29P \quad \text{and} \quad P^\alpha - P^\beta = \delta I - 13P
\]
then \(\alpha + \beta + \gamma – \delta \)is equal to :\\

\textbf{Options:} \\
% Option 
(1) \(40\) \\
% Option 
(2) \(22\) \\
% Option 
(3) \(24\) \\
% Option 
(4) \(18 \)\\
% Correct Answer
\textbf{Correct Answer:} (3) \(24\) \\
% Solution
\noindent \textbf{Solution:} \\

\textbf{Step 1: Understand the given matrix properties.} \\
We are given that \( P^2 = I - P \). Using this, we derive higher powers of \( P \):

\[P^2 &= I - P \\\]
\[P^{\alpha} + P^{\beta} &= \gamma I - 29P \\\]
\[P^{\alpha} - P^{\beta} &= \delta I - 13P \\\]
\[P^4 &= (I - P)^2 = I + P^2 - 2P \\\]
\[&= I + I - P - 2P = 2I - 3P \\\]
\[P^8 &= (P^4)^2 = (2I - 3P)^2 = 4I + 9P^2 - 12P \\\]
\[&= 4I + 9(I - P) - 12P \\\]
\[&= 13I - 21P \\\]
\textbf{Step 2: Simplify the given equations.} \\
Substitute the derived values into the equations and solve for \( \alpha, \beta, \gamma, \delta \):

\[P^6 &= P^4 \cdot P^2 \\\]
\[&= (2I - 3P)(I - P) \\\]
\[&= 2I - 5P + 3P^2 \\\]
\[&= 2I - 5P + 3(I - P) \\\]
\[&= 5I - 8P \\\]
\[P^8 + P^6 &= 18I - 29P \\\]
\[P^8 - P^6 &= 8I - 13P \\\]

From this, we deduce that:

\[\alpha &= 8, \quad \beta = 6 \\\]
\[\gamma &= 18 \\\]
\[\delta &= 8 \\\]

\textbf{Step 3: Calculate the final answer.} \\
\[
\alpha + \beta + \gamma - \delta = 32 - 8 = 24
\]

\textbf{Final Answer:} The value of \( \alpha + \beta + \gamma - \delta \) is 24.
% Quick Tip
\begin{tcolorbox}[colback=white, colframe=black, colbacktitle=white, coltitle=black,
title=QuickTip, width=\textwidth, halign=left]
For matrix equations, understanding the properties of matrix powers can simplify the problem-solving process.
\end{tcolorbox}

\vspace{0.5cm}
\hline
\vspace{0.5cm}
% Question text
\noindent \textbf{7. A plane P contains the line of intersection of the planes \( \Vec{r} \cdot (i + j + k) = 6 \) and \( \Vec{r} \cdot (2i + 3j + 4k) = -5 \). If P passes through the point \( (0, 2, -2) \), then the square of distance of the point \( (12, 12, 18) \) from the plane P is:} \\
\textbf{Options:} \\
% Option 
(1) \(620\) \\
% Option 
(2) \(1240 \)\\
% Option 
(3) \(310\) \\
% Option 
(4) \(155\) \\
% Correct Answer
\textbf{Correct Answer:} (1) \(620\) \\
% Solution
\noindent \textbf{Solution:} \\

\textbf{Step 1: Equation of plane P.} \\
The equation of the plane is:
\[
P_1 + \lambda P_2 = 0
\]
Where \( P_1 = (x + y + z - 6) \) and \( P_2 = (2x + 3y + 4z + 5) \).

\textbf{Step 2: Passing through the point \( (0, 2, -2) \).} \\
Substitute \( (0, 2, -2) \) into the equation:
\[
(-6) + \lambda [3] = 0 \quad \Rightarrow \lambda = 2
\]

\textbf{Step 3: Equation of plane P.} \\
Thus, the equation of the plane is:
\[
5x + 7y + 9z + 4 = 0
\]

\textbf{Step 4: Distance from point \( (12, 12, 18) \).} \\
The distance is given by:
\[
d = \frac{|60 + 84 + 162 + 4|}{\sqrt{25 + 49 + 81}} = \frac{310}{\sqrt{155}}
\]
\[
d^2 = \frac{310 \times 310}{155} = 620
\]

\textbf{Final Answer:} The square of the distance is \( 620 \).
% Quick Tip
\begin{tcolorbox}[colback=white, colframe=black, colbacktitle=white, coltitle=black,
title=QuickTip, width=\textwidth, halign=left]
To find the distance from a point to a plane, use the formula for distance from a point to a plane \( d = \frac{|Ax_1 + By_1 + Cz_1 + D|}{\sqrt{A^2 + B^2 + C^2}} \).
\end{tcolorbox}

\vspace{0.5cm}
\hline
\vspace{0.5cm}

% Question text
\noindent \textbf{8. Let \( f(x) \) be a function satisfying \( f(x) + f(\pi - x) = \pi^2 \), for all \( x \in \mathbb{R} \). Then \( \int_0^\pi f(x) \sin x \, dx \) is equal to:} \\
\textbf{Options:} \\
% Option 
(1) \( \frac{\pi^2}{2} \) \\
% Option 
(2) \( \pi^2 \) \\
% Option 
(3) \( 2\pi^2 \) \\
% Option 
(4) \( \frac{\pi^2}{4} \) \\
% Correct Answer
\textbf{Correct Answer:} (2) \( \pi^2 \) \\
% Solution
\noindent \textbf{Solution:} \\

\textbf{Step 1: Express the integral.} \\
Let:
\[
I = \int_0^\pi f(x) \sin x \, dx
\]

\textbf{Step 2: Apply the King property.} \\
Using the given functional identity, we have:
\[
I = \int_0^\pi f(\pi - x) \sin(\pi - x) \, dx
\]
Since \( \sin(\pi - x) = \sin(x) \), we obtain:
\[
I = \int_0^\pi f(\pi - x) \sin x \, dx
\]

\textbf{Step 3: Add the integrals.} \\
Adding the two expressions for \( I \), we get:
\[
2I = \int_0^\pi [f(x) + f(\pi - x)] \sin x \, dx
\]
Substitute \( f(x) + f(\pi - x) = \pi^2 \):
\[
2I = \int_0^\pi \pi^2 \sin x \, dx
\]
\[
2I = \pi^2 \left[ -\cos x \right]_0^\pi = \pi^2 (2)
\]
\[
I = \pi^2
\]

\textbf{Final Answer:} \( \int_0^\pi f(x) \sin x \, dx = \pi^2 \).
% Quick Tip
\begin{tcolorbox}[colback=white, colframe=black, colbacktitle=white, coltitle=black,
title=QuickTip, width=\textwidth, halign=left]
The King property can simplify integrals involving symmetric functions. Use it to simplify integrals over the interval \( [0, \pi] \).
\end{tcolorbox}

\vspace{0.5cm}
\hline
\vspace{0.5cm}

%




'''

In [50]:
df = extract_latex_to_dataframe(latex_code)
df

,S.no,Question text,Option_1_en,Option_2_en,Option_3_en,Option_4_en,Correct Answer,Solution,Quick Tip
0,1,"If gcd(m, n) = 1 and $1^2-2^2+3^2-4^2+\cdots+(...",\(180\),\(220\),\(200\),\(240\),(4) \(240\),\\\textbf{Step 1: Simplify the series.} \\Usin...,Using properties of sequences and algebraic ma...
1,2,The area bounded by the curves $y = |x - 1| + ...,\(5\),\(4 \),\(6\),\(3 \),(2) \(4 \),\\\textbf{Step 1: Understand the geometric int...,"For areas between curves, integrate the differ..."
2,3,"For the system of equations $x + y + z = 6$, $...",\(System has a unique solution for $\alpha$ = ...,\(System has a unique solution for $\alpha = -...,"\(System has no solution for $\alpha = 3$, $\b...",\(System has infinitely many solutions for $\a...,(1) \(System has a unique solution for $\alpha...,\\\textbf{Step 1: Calculate the determinant $\...,Cramer's rule is pivotal for determining the n...
3,4,Among the statements: \item (S1): $(p \Rightar...,NaN,NaN,NaN,NaN,(3) \(Neither (S1) and (S2) is True\),\\\textbf{Step 1: Analyze S1.} \\Construct the...,Truth tables are essential for analyzing logic...
4,5,Evaluate the limit \\ \[lim_{n->\infty \{(2^{{...,\( \frac{1}{\sqrt{2}} \),\( \sqrt{2} \),\(1\),\(0 \),(4) \(0 \),\\\textbf{Step 1: Analyze the behavior of each...,"\begin{tcolorbox}[colback=white, colframe=blac..."
5,6,Let P be a square matrix such that \( P^2 = I ...,\(40\),\(22\),\(24\),\(18 \),(3) \(24\),\\\textbf{Step 1: Understand the given matrix ...,"\begin{tcolorbox}[colback=white, colframe=blac..."
6,7,A plane P contains the line of intersection of...,\(620\),\(1240 \),\(310\),\(155\),(1) \(620\),\\\textbf{Step 1: Equation of plane P.} \\The ...,"\begin{tcolorbox}[colback=white, colframe=blac..."
7,8,Let \( f(x) \) be a function satisfying \( f(x...,\( \frac{\pi^2}{2} \),\( \pi^2 \),\( 2\pi^2 \),\( \frac{\pi^2}{4} \),(2) \( \pi^2 \),\\\textbf{Step 1: Express the integral.} \\Let...,"\begin{tcolorbox}[colback=white, colframe=blac..."
8,9,If the coefficients of \( x^7 \) in \( (ax^2 +...,\( 64ab = 243 \),\( 32ab = 729 \),\( 729ab = 32 \),\( 243ab = 64 \),(3) \( 729ab = 32 \),\\\textbf{Step 1: Find the coefficient of \( x...,"\begin{tcolorbox}[colback=white, colframe=blac..."
9,10,If the tangents at the points P and Q on the c...,\( 1\),\(2 \),\(4\),\(3\),(3) \( \frac{5}{8} \),\\\textbf{Step 1: Equation of the circle.} \\T...,"\begin{tcolorbox}[colback=white, colframe=blac..."


In [32]:
df.to_excel("out.xlsx", index=False)
